In [42]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

In [2]:
df=pd.read_csv("/content/mumbai.csv")
df.head()

,price,Address,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,Status,neworold,parking,Furnished_status,Lift,Landmarks,type_of_building,desc,Price_sqft
0,22400000.0,"Chembur, Mumbai - Harbour Line, Maharashtra",629.0,19.032800,72.896357,2.0,2.0,NaN,Under Construction,New Property,NaN,NaN,2.0,NaN,Flat,\n\n\n Right fr...,35612.082671
1,35000000.0,"Chembur, Mumbai - Harbour Line, Maharashtra",974.0,19.032800,72.896357,3.0,2.0,NaN,Under Construction,New Property,NaN,NaN,2.0,NaN,Flat,\n\n\n Right fr...,35934.291581
2,31700000.0,"Ghatkopar East, Mumbai - Central Line, Maharas...",968.0,19.085600,72.909277,3.0,3.0,NaN,Under Construction,New Property,1.0,NaN,2.0,NaN,Flat,\n \n \n...,32747.933884
3,18700000.0,"Plot Number 283, Road Number 3, Jawahar Nagar,...",629.0,19.155756,72.846862,2.0,2.0,2.0,Ready to Move,New Property,1.0,NaN,3.0,Laxmi Callista,Flat,\n\n\n Welcome ...,29729.729730
4,13500000.0,"malad east, Malad East, Mumbai - North Mumbai,...",1090.0,19.177555,72.849887,2.0,2.0,NaN,NaN,New Property,1.0,Unfurnished,2.0,near uper govind nagar,Flat,\n \n \n...,12385.321101


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6255 entries, 0 to 6254
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             6255 non-null   float64
 1   Address           6255 non-null   object 
 2   area              6255 non-null   float64
 3   latitude          6255 non-null   float64
 4   longitude         6255 non-null   float64
 5   Bedrooms          6255 non-null   float64
 6   Bathrooms         6255 non-null   float64
 7   Balcony           2376 non-null   float64
 8   Status            5774 non-null   object 
 9   neworold          6255 non-null   object 
 10  parking           2129 non-null   float64
 11  Furnished_status  3600 non-null   object 
 12  Lift              1678 non-null   float64
 13  Landmarks         2617 non-null   object 
 14  type_of_building  6255 non-null   object 
 15  desc              6255 non-null   object 
 16  Price_sqft        6255 non-null   float64


In [4]:
columns_to_drop = ['Balcony', 'parking', 'Lift','Landmarks','Address','desc']


df = df.drop(columns=columns_to_drop,)

drop :7,10,12,13
complete: 8,11

In [5]:
most_common_Status=df['Status'].mode()[0]

df['Status'].fillna(most_common_Status, inplace=True)



In [6]:
most_common_Furnished_status=df['Furnished_status'].mode()[0]
df['Furnished_status'].fillna(most_common_Furnished_status, inplace=True)

In [7]:
df.head()

,price,area,latitude,longitude,Bedrooms,Bathrooms,Status,neworold,Furnished_status,type_of_building,Price_sqft
0,22400000.0,629.0,19.032800,72.896357,2.0,2.0,Under Construction,New Property,Unfurnished,Flat,35612.082671
1,35000000.0,974.0,19.032800,72.896357,3.0,2.0,Under Construction,New Property,Unfurnished,Flat,35934.291581
2,31700000.0,968.0,19.085600,72.909277,3.0,3.0,Under Construction,New Property,Unfurnished,Flat,32747.933884
3,18700000.0,629.0,19.155756,72.846862,2.0,2.0,Ready to Move,New Property,Unfurnished,Flat,29729.729730
4,13500000.0,1090.0,19.177555,72.849887,2.0,2.0,Ready to Move,New Property,Unfurnished,Flat,12385.321101


In [8]:
df.isnull().sum()

price               0
area                0
latitude            0
longitude           0
Bedrooms            0
Bathrooms           0
Status              0
neworold            0
Furnished_status    0
type_of_building    0
Price_sqft          0
dtype: int64

In [9]:
df.describe()

,price,area,latitude,longitude,Bedrooms,Bathrooms,Price_sqft
count,6.255000e+03,6255.000000,6255.000000,6255.000000,6255.000000,6255.000000,6255.000000
mean,2.684187e+07,1185.625260,19.156772,72.868642,2.452278,2.425580,21422.923601
std,2.788831e+07,636.973259,0.109447,0.058614,0.749534,0.762604,12698.478665
min,1.500000e+06,503.000000,18.873713,72.754080,2.000000,0.000000,1500.000000
25%,1.300000e+07,846.500000,19.081509,72.832930,2.000000,2.000000,13709.934735
50%,1.900000e+07,1000.000000,19.147269,72.858019,2.000000,2.000000,19285.714286
75%,3.000000e+07,1300.000000,19.213054,72.884691,3.000000,3.000000,26500.000000
max,3.600000e+08,8000.000000,19.476239,73.197823,10.000000,10.000000,119047.619048


outliers:
area range from 393 to 1753.5
latitude range        


In [10]:
numericalcolumns = ['price', 'area', 'latitude', 'longitude', 'Bedrooms', 'Bathrooms', 'Price_sqft']


In [11]:

def remove_outliers_iqr(data, columns):
    for column in columns:
        Q3 = data[column].quantile(0.75)
        Q1 = data[column].quantile(0.25)
        IQR = Q3 - Q1

        lower_range = Q1 - 1.5 * IQR
        upper_range = Q3 + 1.5 * IQR

        data = data[(data[column] >= lower_range) & (data[column] <= upper_range)]

    return data


In [12]:
df = remove_outliers_iqr(df, numericalcolumns)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3736 entries, 0 to 6254
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   price             3736 non-null   float64
 1   area              3736 non-null   float64
 2   latitude          3736 non-null   float64
 3   longitude         3736 non-null   float64
 4   Bedrooms          3736 non-null   float64
 5   Bathrooms         3736 non-null   float64
 6   Status            3736 non-null   object 
 7   neworold          3736 non-null   object 
 8   Furnished_status  3736 non-null   object 
 9   type_of_building  3736 non-null   object 
 10  Price_sqft        3736 non-null   float64
dtypes: float64(7), object(4)
memory usage: 350.2+ KB


In [14]:
df['Status'].unique().sum()

'Under ConstructionReady to Move'

In [15]:
df['neworold'].unique()

array(['New Property', 'Resale'], dtype=object)

In [16]:
df['Furnished_status'].unique()

array(['Unfurnished', 'Semi-Furnished', 'Furnished'], dtype=object)

In [17]:
df['type_of_building'].unique()

array(['Flat', 'Individual House'], dtype=object)

In [18]:
categorical_cols = [ 'Status', 'neworold', 'Furnished_status', 'type_of_building']

In [19]:
numerical_cols = ['area', 'latitude', 'longitude', 'Bedrooms', 'Bathrooms', 'Price_sqft']

In [20]:
label_encoder = LabelEncoder()

In [21]:
df['Status'] = label_encoder.fit_transform(df['Status'])
df['neworold'] = label_encoder.fit_transform(df['neworold'])
df['Furnished_status'] = label_encoder.fit_transform(df['Furnished_status'])
df['type_of_building'] = label_encoder.fit_transform(df['type_of_building'])


In [22]:
df.head()

,price,area,latitude,longitude,Bedrooms,Bathrooms,Status,neworold,Furnished_status,type_of_building,Price_sqft
0,22400000.0,629.0,19.032800,72.896357,2.0,2.0,1,0,2,0,35612.082671
1,35000000.0,974.0,19.032800,72.896357,3.0,2.0,1,0,2,0,35934.291581
3,18700000.0,629.0,19.155756,72.846862,2.0,2.0,0,0,2,0,29729.729730
4,13500000.0,1090.0,19.177555,72.849887,2.0,2.0,0,0,2,0,12385.321101
5,13000000.0,630.0,19.148058,72.937725,2.0,2.0,0,0,2,0,20634.920635


In [23]:
X = df[['area', 'latitude', 'longitude', 'Bedrooms', 'Bathrooms', 'Price_sqft','Status','neworold','Furnished_status','type_of_building']]
y = df['price']


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [38]:
linear_regression_model = LinearRegression()
linear_regression_model.fit(X_train, y_train)
linear_regression_score = linear_regression_model.score(X_test, y_test)

print(f"Linear Regression R^2 Score: {linear_regression_score}")


Linear Regression R^2 Score: 0.9611443889520613


In [26]:
knn_model = KNeighborsRegressor()



In [27]:
X_knn = df[['area', 'latitude', 'longitude', 'Bedrooms', 'Bathrooms', 'Price_sqft',
            'Status', 'neworold', 'Furnished_status', 'type_of_building']]
y_knn = df['price']


In [32]:
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X_knn, y_knn, test_size=0.2, random_state=42)

knn_model.fit(X_train_knn, y_train_knn)
knn_score = knn_model.score(X_test_knn, y_test_knn)

print(f"KNN R^2 Score: {knn_score}")

KNN R^2 Score: 0.9636174219509792
